In [1]:
import torch
import torch.nn as nn #contains different classess that help you build neural network models. like activation fun forward
import torch.nn.functional as F #used for convolution
import torchvision #for datasets
import torchvision.transforms as transforms #used for image transforms to tensor
import matplotlib.pyplot as plt #to plot
import numpy as np

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_size=784
hi_size=100
num_classes=10
num_epochs=100
batch_size=512
learning_rate=0.04
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)#downloads the data 
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
#Combines a dataset and a sampler, and provides an iterable over the given datas
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)
print(len(train_loader))
examples= iter(train_loader) # returns an iterator for the given object
samples,labels=examples.next()
print(samples.shape,labels.shape)

In [ ]:
class ConvNet(nn.Module): #class for making forward prob
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1=nn.Conv2d(1,6,5)# input(channels) ,output, kernal sizes
    self.pool= nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*4*4,120)#16*4*4 do conv1,2 and pool to get the last layer length
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

    
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*4*4)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x
    


model= ConvNet().to(device)

criterion=nn.CrossEntropyLoss() #computes loss

optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
#Optimizers are algorithms or methods used to change the attributes of the neural network such as weights and learning rate to reduce the losses
#training loop
train_losses = []
train_counter = []
test_losses = []
n_steps=len(train_loader)
for epoch in range(num_epochs):#will loop over epochs
  for i,(images,labels) in enumerate(train_loader): #enumerate loops over the given data and gives labels i mean changes them after every iter
    images=images.to(device)
    labels=labels.to(device)
    #forward
    outputs=model(images)
    loss=criterion(outputs,labels)
    train_losses.append(loss.item())

    #backward
    optimizer.zero_grad()#empty grad
    loss.backward()#does backprop
    optimizer.step()#upates the parameters for us
    if (i+1) % 100 == 0:
      print(f'Train Epoch: {epoch+1}/{num_epochs}, step{i+1}/{n_steps}, loss ={loss.item():.4f}')



def test(): 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
      images=images.to(device)
      labels=labels.to(device)
      outputs = model(images)

      _,predictions = torch.max(outputs, 1) #torch.max function will return the value and the index and we are interested in the actual index
      n_samples += labels.shape[0]
      n_correct += (predictions == labels).sum().item()
  
    acc = 100* n_correct / n_samples
    print(f'Test accuracy = {acc:.4f}')

def train(): 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in train_loader:
      images=images.to(device)
      labels=labels.to(device)
      outputs = model(images)

      _,predictions = torch.max(outputs, 1) #torch.max function will return the value and the index and we are interested in the actual index
      n_samples += labels.shape[0]
      n_correct += (predictions == labels).sum().item()
  
    acc = 100* n_correct / n_samples
    print(f'Training accuracy = {acc:.4f}')

train()
test()

In [ ]:
plt.plot(train_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.title("Learning rate 0.04")